In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url_habr ='https://habr.com/ru/companies/'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.37",
    "Accept": "application/json, text/plain, */*"
} 

In [3]:
response = requests.get(url_habr, headers=headers)
soup = BeautifulSoup(response.text, "lxml")
pages_count = int(soup.find_all("a", class_="tm-pagination__page")[-1].text)
pages_count

17

Для проверки используется только 2стр.

In [4]:
data = pd.DataFrame(columns=['name company', 'count blogs in may', 'badges', 'rating', 'subscribers'])
#for page in range(1, pages_count + 1):
for page in range(1, 3):
    url = f'https://habr.com/ru/companies/page{page}/'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "lxml")
    companies = soup.find_all("div", class_="tm-companies__item tm-companies__item_inlined")
    print('page:', page, '\n')
    
    for company in companies:
        # Парсинг значков
        company_badges = []
        company_link = company.find("a", class_="tm-company-snippet__title").get('href')
        response = requests.get(f'https://habr.com{company_link}', headers=headers)
        soup = BeautifulSoup(response.text, "lxml")
        badges = soup.find_all("li", class_="tm-badges__item")
        if badges == []:
                company_badges = 'Нет'
        else:
            for badge in badges:
                company_badges.append(badge.find("span").text)

        # Парсинг постов в мае
        response = requests.get(f'https://habr.com{company_link[:-9]}/articles/', headers=headers)
        soup = BeautifulSoup(response.text, "lxml")
        blogs = soup.find_all("div", class_='tm-article-snippet tm-article-snippet')
        count_blogs_in_may = 0
        company_blogs_in_may = []
        for blog in blogs:
            blog_time = blog.find("span", class_='tm-article-datetime-published').text
            if blog_time.find('мар') != -1:
                count_blogs_in_may += 1
                company_blogs_in_may.append('https://habr.com' + blog.find('a', class_='tm-title__link').get('href'))

        # Парсинг Имени
        company_title = company.find("a", class_="tm-company-snippet__title").text

        # Парсинг кол-во рейтинга
        company_counter_rating = float(company.find("span", 
                                                    class_="tm-companies__score-counter tm-companies__score-counter_rating")
                                                    .text.strip())

        # Парсинг подписчиков
        company_subscribers = company.find("span", class_="tm-companies__score-counter tm-companies__score-counter_subscribers")\
                                                                                                                   .text.strip()
        if company_subscribers.find('K') != -1:
            company_subscribers = float(company_subscribers.replace('K', '')) * 1000
        # Вывод

        company_link = f'https://habr.com{company_link}'
        data.loc[len(data.index)] = [company_title, count_blogs_in_may, company_badges, 
                                     company_counter_rating, company_subscribers]
        print('company:', company_title, company_link)
        print('count blogs in may:', count_blogs_in_may, ', links:', company_blogs_in_may)
        print('badges:', company_badges)
        print('rating:', company_counter_rating)
        print('subscribers:', company_subscribers)
        print('\n')
        
data

page: 1 

company: RUVDS.com https://habr.com/ru/companies/ruvds/profile/
count blogs in may: 4 , links: ['https://habr.com/ru/companies/ruvds/articles/725502/', 'https://habr.com/ru/companies/ruvds/articles/726102/', 'https://habr.com/ru/companies/ruvds/articles/724822/', 'https://habr.com/ru/companies/ruvds/articles/721686/']
badges: ['Лучший блог компании 2020']
rating: 2237.39
subscribers: 74000.0


company: Timeweb Cloud https://habr.com/ru/companies/timeweb/profile/
count blogs in may: 7 , links: ['https://habr.com/ru/companies/timeweb/articles/723474/', 'https://habr.com/ru/companies/timeweb/articles/725376/', 'https://habr.com/ru/companies/timeweb/articles/720510/', 'https://habr.com/ru/companies/timeweb/articles/724632/', 'https://habr.com/ru/companies/timeweb/articles/724146/', 'https://habr.com/ru/companies/timeweb/articles/724158/', 'https://habr.com/ru/companies/timeweb/articles/723332/']
badges: Нет
rating: 1848.18
subscribers: 1200.0


company: Selectel https://habr.com/

,name company,count blogs in may,badges,rating,subscribers
0,RUVDS.com,4,[Лучший блог компании 2020],2237.39,74000.0
1,Timeweb Cloud,7,Нет,1848.18,1200.0
2,Selectel,13,Нет,1804.51,40000.0
3,VK,12,[Лучший блог компании 2020],880.35,169000.0
4,OTUS,0,Нет,774.14,105000.0
5,Яндекс,4,[Лучший блог компании 2020],718.23,166000.0
6,Open Data Science,2,Нет,511.69,9700.0
7,FirstVDS,15,Нет,501.64,24000.0
8,ГК ЛАНИТ,4,Нет,471.58,55000.0
9,getmatch,7,Нет,465.13,988
